# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. 
Цель: разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.




Описание данных:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

## Шаг. Откройте файл с данными и изучите общую информацию

In [1]:
# для дальнейшего удобства загружаем все необходимые библиотеки в одной ячейке
# библиотека, которая необходима для лемматизации

from pymystem3 import Mystem

#библиотека для подсчета количества лемм
from collections import Counter

import pandas as pd #импортируем библиотеку pandas
data = pd.read_csv('/datasets/data.csv') #читаем файл с данными и сохраняем его в переменную data


In [2]:
data.info() #получение общей информации о данных в таблице
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
data['gender'].value_counts() 
#после ревью был обнаружен 3 гендер, странно, либо кто-то пошутил, либо клиент банка является небинарной личностью

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [4]:
data['children'].value_counts() 

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Еще один артефакт: отрицательное значение детей, а у кого-то их вообще 20. Думаю надо исправить

**Вывод**

В таблице 12 столбцов. В них встречается 3 типа данных:
* float64
* int64
* object(5) 

Наш датасет состоит из 21525 строк и 12 столбцов.
По первым 5-ти строкам сразу видим, что присутствует отрицательный трудовой стаж, разный регистр в образовании, совпадение цели кредита (что не является проблемой, а поможет при категоризации данных), нужно будет только применить метод лемматизации.

В колонке дни стажа работы присутствуют 6-ти значные значения, это говорит о том, что люди работали около 1000 лет). Тут явно какая то ошибка, возможно написали в часах, или сотрудник банка торопился и случайно нажал лишнюю клавишу - поправим. 
Также обнаружен 3 гендер, думаю можно попросить кредитный отдел проверить правильность заполнения информации, но я за самоопределение человека и поэтому не заменяю его на мужской или женский.

* Количественные переменные в стобцах: children, days_employed, dob_years, total_income

* Категориальные: education, education_id, family_status,	family_status_id, gender, income_type, debt, purpose

* Логические переменные отсутствуют так как нет столбцов со значениями True/False

**Над чем работаем:**
* пропущенные значения в days_employed и total_income
* некорректны тип данных в столбце days_employed
* дубликаты в purpose и education
* большие значения в days_employed, которые показывают опыт работы скорее в часах или минутах
* третий гендер
* отрицательное количество детей, а также количество детей, равное 20 (сильно много)

## Шаг. Предобработка данных

### Работа с артефактами

**Работа с количеством детей**


In [5]:
#Проверим, какое количество и как часто встречается в наших данных
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

* Обнаружено, что есть -1 ребенок, а также 20 детей в семье. Предполагаю, что в данных ошибка, появившаяся в процессе заполнения данных о клиенте. Предлагаю изменить отрицательное значение на положительное, а 20 детей на 2, что более логично и реально.

* Для того, чтобы изменить отрицательное значение на положительное, воспользуемся ранее функцией по замене на модуль числа

In [6]:
# Функция для замены отрицательного числа на модуль этого числа
def modul_days(days):
    if days<0:
        days = abs(days)
    return days

In [7]:
# применяем функцию modul_days к столбцу 'children'
data['children'] = data['children'].apply(modul_days)
# сразу проверим как обстоят дела, выведем информацию с помощью value_counts()
data['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

**Отлично**, осталось изменить количество детей с 20 до 2. Для этого воспользуемся методом .replace()

In [8]:
# применяем метод .replace() к столбцу 'children'
data['children'] = data['children'].replace(20, 2)

Проверим, как после проделанной работы изменилось распределение детей в данных

In [9]:
# для этого воспользуемся методом value_counts()
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**ВЫВОД**:

В столбце `children` приведено количество детей к более реальным показателям, заменили отрицательные значения.

### Обработка пропусков

In [10]:
#посчитаем количество пропусков
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

* Видим, что пропущены значения в столбцах о стаже и заработке. Проверим, связаны ли они между собой или значения пропущены в разных строках?

In [11]:
data_null = data[data['days_employed'].isna() & data['total_income'].isna()]
display(data_null.head(10)) #просмотрим первые 5 строк
print('Количество пропущенных значений', data_null.shape[0]) #найдем размер новой таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Количество пропущенных значений 2174


Пропущенные значения в трудовом стаже 'days_employed' связаны с пропущенными значениями в зарплате 'total_income'. Это немного упрощает дальнейшую работу.

Возможная причина: ошибка при выгрузке данных. Вообще можно попросить отправить корректные данные, но мы работаем с тем, что есть. Также можно отметить, что эти пропуски у разной категории занятости - сотрудники, пенсионеры, госслужащие и т.п.

Пропуски в трудовом стаже в колонке 'days_employed' не оказывают влияния на наше исследование, а вот пропуск значений в колонке уровня зарплат 'total_income' может привести к неточности анализа, ведь у нас есть вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?"

Что делаем?

* Удалять пропущенные значения в 'total_income', также заменять их пустыми строками мы не будем, так как они составляют около 10% от объема данных.

* Следователь, их необходимо заполнить медианным или средним значением.

In [12]:
#создаю функцию, которая проверить зависимость появления пропусков от других данных
#она группирует по категориям столбца и возвращает процент пропуска по категории
#purpose выбрана, так как в ней нет пропусков

def percent_null(column):
    f_category = data.pivot_table(index=[column], values=['purpose', 'days_employed'], aggfunc=['count'])
    f_category['perc_null'] = (1-(f_category['count']['days_employed']/f_category['count']['purpose']))*100
    f_category['perc_null'] = round(f_category['perc_null'], 1)
    return f_category 

In [13]:
#проверим работу функции на столбце с детьми
percent_null('children')

count         perc_null
         days_employed purpose          
children                                
0                12710   14149      10.2
1                 4387    4865       9.8
2                 1918    2131      10.0
3                  294     330      10.9
4                   34      41      17.1
5                    8       9      11.1

**Пропуски равномерно разбросаны по всем категориям, в среднем по 10. Исключение это 17% у людей с 4 детьми, учитывая что это 41 значение из 21 тыс. Также наблюдаю меньшую выборку у людей с 5 детьми.**

In [14]:
#оценим пропуски в категории по столбцу 'family_status'
percent_null('family_status')

count         perc_null
                      days_employed purpose          
family_status                                        
Не женат / не замужем          2525    2813      10.2
в разводе                      1083    1195       9.4
вдовец / вдова                  865     960       9.9
гражданский брак               3735    4177      10.6
женат / замужем               11143   12380      10.0

**Аналогичное представление. Пропуски равномерно разбросаны по всем категориям**

**Обнаружено, что пропуски имеются в двух колонках 'days_employed' и 'total_income'. Они находятся в одних и тех же местах в 2174 строках. Распреление пропусков в соответствии с различными группа показало, что они распределены внутри категорий столбцов также, как в общем датасете (около 10%). Какая-либо закономерность появления пропущенных значений не прослеживается. Они являются симметричными и скорее всего полностью случайными, которые ни от чего не зависят. Предпологаю, что это ошибка в выгрузке данных, ну или так совпало, что кредитный менеджер забывал их заполнять в какие то дни)**

In [15]:
# определяем среднее значение зарплаты
total_income_mean = data['total_income'].mean()
print(total_income_mean)

167422.30220817294


In [16]:
# определяем медианное значение зарплаты
total_income_median = data['total_income'].median()
print(total_income_median)

145017.93753253992


In [17]:
# в дополнение определим максимальный и минимальный уровень зарплат
print('Максимальный уровень зарплаты', data['total_income'].max())
print('Минимальный уровень зарплаты', data['total_income'].min())

Максимальный уровень зарплаты 2265604.028722744
Минимальный уровень зарплаты 20667.26379327158


Получается очень широкий коридор уровня зарплаты: от 20 тыс. до 2.2 млн. Учитывая, что пропущенные значения принадлежат и пенсионерам и обычным сотрудникам, остановлюсь на том, что пропущенные значения зп заполню медианным значением.

In [18]:
# заменяем пропущенные значения в столбце 'total_income' медианным значением
data['total_income'] = data['total_income'].fillna(total_income_median)

In [19]:
# заменяем пропущенные значения в столбце 'days_employed' на 0.
data['days_employed'] = data['days_employed'].fillna(0)

In [20]:
# проверяем, как сработали оба дейсвия замены
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Устранены следующие пропуски:
* В столбце уровня заработной планы 'total_income' пропущенные данные заменены медианным значением с связи с широким диапазоном полученных значений
* Пропущенные дни стажа в колонке 'days_employed' заменены на нулевые значения, так как не учиваются в нашем исследовании

### Замена типа данных

В первом шаге при получении общей информации о полученных данных обнаружено, что показатели трудового стажа в днях имеют нецелочисленный тип данных:
- days_employed       19351 non-null float64
Также я принял решение округлить доходы заявителей в банк. Это будет нужно при дальнейшей категоризации

In [21]:
#изменение типа данных столбца 'days_employed' с float на int с помощью метода astype()
data['days_employed'] = data['days_employed'].astype('int')
#изменение типа данных столбца 'total_income' с float на int с помощью метода astype()
data['total_income'] = data['total_income'].astype('int')

data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Тип данных изменен. Но можно заметить отрицательные значения, которые нехарактерны для "дней". Для этого напишем фукцию, которая поможет заменить все отрицательные значения на положительные.

In [22]:
# Функция для замены отрицательного числа на модуль этого числа
def modul_days(days):
    if days<0:
        days = abs(days)
    return days

Пробуем применить данную функцию к столбцу 'days_employed' c помощью метода .apply()

In [23]:
data['days_employed'] = data['days_employed'].apply(modul_days)

In [24]:
# Проверим сколько отрицательных значений осталось после применения функции
print('Количество отрицательных значений дней в данных', data[data['days_employed'] < 0].shape[0])

Количество отрицательных значений дней в данных 0


**Вывод**


* В столбце 'days_employed' изменен вещественный тип данный на целочисленный для учета колиства дней. Заменены отрицательные значения на модуль числа.
* В столбце 'children' приведено количество детей к более реальным показателям, заменили отрицательные значения.

### Обработка дубликатов

Дубликаты бывают неявные и явные. Предлагаю начать с неявных дубликатов, которые уже видно в Шаге 1. В колонке степени образования видно, что они записаны разным регистром. Проверим уникальные значения в этой колонке.

#### Обработка неявных дубликатов

In [25]:
#определим уникальные значения в колонке образования
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Такая проблема обычно появляется при заполнении данных, но это можно поправить.
Лишние пробелы отсутствуют, поэтому можно привести все к одному нижнему регистру и посмотреть на результат

In [26]:
# приведем значения в колонке 'education' к нижнему регистру 
data['education'] = data['education'].str.lower()

Проверим, сколько значений осталось в колонке образования

In [27]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

**Отлично.** Такой результат нас устраивает: от неоконченного высшего до ученой степени, и ни одной записи с разным регистром

#### Обработка явных дубликатов

Для начала определим количество явных дубликатов в наших данных

In [28]:
# используем метод duplicated() для определения дубликатов и метод sum() для подсчета их количество
data.duplicated().sum()

71

In [29]:
# выведем дубликаты в таблицу для просмотра
data[data.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,0,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для семьи
3290,0,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
4182,1,0,34,высшее,0,гражданский брак,1,F,сотрудник,0,145017,свадьба
4851,0,0,60,среднее,1,гражданский брак,1,F,пенсионер,0,145017,свадьба
5557,0,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,0,64,среднее,1,женат / замужем,0,F,пенсионер,0,145017,дополнительное образование
21032,0,0,60,среднее,1,женат / замужем,0,F,пенсионер,0,145017,заняться образованием
21132,0,0,47,среднее,1,женат / замужем,0,F,сотрудник,0,145017,ремонт жилью
21281,1,0,30,высшее,0,женат / замужем,0,F,сотрудник,0,145017,покупка коммерческой недвижимости


In [30]:
print('Количество дубликатов от общего датасета составляет {:.2%}'.format(71/21525))

Количество дубликатов от общего датасета составляет 0.33%


Явно видно, что дубликаты определились только по стоблцу 'total_income', значения в котором мы заполняли медианной зарплатой, а может быть это вообще ошибка банка и операционист сам выбирал их уровень зарплат. Исходя из того, что количесто дубликатов составляет всего 0,33% от общего количества данных предлагаю удалить данные дубликаты.


In [31]:
# удаляем дубликаты методом drop_duplicates вместе с присвоением новых индексов
data = data.drop_duplicates().reset_index(drop=True)

In [32]:
# проверяем, сколько дубликатов осталось после их удаления
data.duplicated().sum()

0

**От дубликатов избавились**

**Вывод**
В процессе работы данных я избавился от неявных и явных дубликатов:
* неявные дубликаты были представлены в колонке уровня образования, записанные разным регистром. Устранить данную неточность помогло присвоение нижнего регистра всей колонке;
* явные дубликаты представлены одинаковым уровнем дохода у разных категорий заявителей. Это либо неточность, вызванная заполнением пропусков медианной зарплатой, либо ошибка операциониста, заполнявшего сведения в банке. Явные дублика составляют около 0,3% от всего набора данных, поэтому их удаление не должно привести к неточности в исследовании.

## Лемматизация

**Для того, чтобы выделить основные категории целей кредита, необходимо пройти процесс лемматизации. Это приведение слова к его словарной форме.**


In [33]:
# посмотрим какие цели кредита представлены в нашей таблице и как часто

display(data['purpose'].value_counts())

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Основные запросы населения - это:
1) процессы, связанные с недвижимостью (приобретение жилья, в т.ч. под сдачу, ремонт, новое строительство и т.п.)

2) процессы, связанные с автомобилем (покупка нового или б/у, и т.п.)

3) получение образования

4) проведение свадьбы

Будем работать в этом направлении.

In [34]:
# Просмотр уникальных целей кредита
credit_purpose = data['purpose'].sort_values().unique()
display(credit_purpose)


array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

Для удобства создадим строку уникальных значений, как в практической части курса

In [35]:
# строку из списка можно сделать с помощью метода join()
purposes = ' '.join(credit_purpose) 
print(purposes)

автомобили автомобиль высшее образование дополнительное образование жилье заняться высшим образованием заняться образованием на покупку автомобиля на покупку подержанного автомобиля на покупку своего автомобиля на проведение свадьбы недвижимость образование операции с жильем операции с коммерческой недвижимостью операции с недвижимостью операции со своей недвижимостью покупка жилой недвижимости покупка жилья покупка жилья для сдачи покупка жилья для семьи покупка коммерческой недвижимости покупка недвижимости покупка своего жилья получение высшего образования получение дополнительного образования получение образования приобретение автомобиля профильное образование ремонт жилью свадьба свой автомобиль сделка с автомобилем сделка с подержанным автомобилем строительство жилой недвижимости строительство недвижимости строительство собственной недвижимости сыграть свадьбу


* Для лемматизации воспользуемся библиотекой с функцией лемматизации на русском языке — pymystem3

In [36]:
# определяем леммы нашего списка

m = Mystem() 
lemmas = m.lemmatize(purposes)
print(lemmas) 

['автомобиль', ' ', 'автомобиль', ' ', 'высокий', ' ', 'образование', ' ', 'дополнительный', ' ', 'образование', ' ', 'жилье', ' ', 'заниматься', ' ', 'высокий', ' ', 'образование', ' ', 'заниматься', ' ', 'образование', ' ', 'на', ' ', 'покупка', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'недвижимость', ' ', 'образование', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'операция', ' ', 'с', ' ', 'недвижимость', ' ', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'покупка', ' ', 'жилье', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'сдача', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'покупка', ' ', 'недвижимость', ' ', 'покупка',

* Теперь посчитаем числа их упоминаний. Для этого вызывают специальный контейнер Counter из модуля collections.

In [1]:
print(Counter(lemmas)) 

NameError: name 'Counter' is not defined

В результате проделланных операций мы можем для каждой категории целей кредита создать собственный словарь на основе лемматизации.
1. Процессы, связанные с **недвижимостью**: 'недвижимость', 'жилье', 'строительство', 'коммерческий', 'жилой', 'сдача', 'ремонт';

2. Процессы, связанные с **автомобилем**: 'автомобиль', 'подержать';

3. Получение **образования**: 'образование', 'дополнительный', 'высокий', 'профильный';

4. Проведение **свадьбы**: 'сыграть', 'проведение', 'свадьба'

В данные категории не включены такие слова как **покупка**, **получение** и т.п., а также предлоги, потому что они перекликаются в разных целях и запутают процесс

**Вывод**

В этом разделе была проведена лемматизация - нахождение словарной формы слова. Она необходима для следующего этапа - котегорацизации данных. 

Результатом стали 4 основные категории целей получения кредита и присвоение каждой категории определенного списка:
1. Процессы, связанные с недвижимостью

2. Процессы, связанные с автомобилем

3. Получение образования

4. Проведение свадьбы

## Категоризация данных

Для каждой категории целей нужно создать список, для того, чтобы значение в ячейке можно было сравнить с набором, соответствующим определенной категории и присвоить этому значению категорию. 
Значение категории будет отображено в новой колонке.

In [38]:
# каждой категории создаем список
# Процессы, связанные с недвижимостью
the_property = [ 'недвижимость', 'жилье', 'строительство', 'коммерческий', 'жилой', 'сдача', 'ремонт']
# Процессы, связанные с автомобилем
cars = ['автомобиль', 'подержать']
# Получение образования
education = ['образование', 'дополнительный', 'высокий', 'профильный']
# Проведение свадьбы
wedding = ['сыграть', 'проведение', 'свадьба']

**Итак, напишем функцию, которая принимает значение в столбце 'purpose', далее его лемматизирует, и если оно совпадает с каким-либо списком, то функция присваивает ему название категории, к которой относится данный список**

In [39]:
def categories(purpose):
    lemmas = m.lemmatize(purpose)
    for lema in lemmas:
        if lema in the_property:
            return 'процессы, связанные с недвижимостью'
        if lema in cars:
            return 'процессы, связанные с автомобилем'
        if lema in education:
            return 'получение образования'
        if lema in wedding:
            return 'проведение свадьбы'

Создадим новый столбец 'target_category' в который попадут новые значения.
Для этого нужен метод apply(): он берёт значения столбца датафрейма и применяет к ним функцию из своего аргумента. Здесь apply() следует вызвать для столбца 'purpose', так как в нём содержатся данные, которые функция примет на вход. Аргументом метода станет сама функция categories.

In [40]:
# создаем новый столбец и применяем функцию
data['target_category'] = data['purpose'].apply(categories)


In [41]:
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,target_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"процессы, связанные с недвижимостью"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"процессы, связанные с автомобилем"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"процессы, связанные с недвижимостью"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,получение образования
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,проведение свадьбы


* Для дальнейше работы необходимо провести категоризацию по уровню дохода

Для данной цели воспользуемся методом **describe()**, который покажет нам в каком процентном соотношении распределены доходы в нашем датасете

In [42]:
#применяем метод describe() и сохраняем полученный series в money_cat
money_cat = data.describe()['total_income']
display(money_cat)
display(money_cat['min'])


count    2.145400e+04
mean     1.652256e+05
std      9.802102e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.450170e+05
75%      1.958132e+05
max      2.265604e+06
Name: total_income, dtype: float64

20667.0

In [43]:
# по аналогии с предыдущей функцией напишем функцию categories_money, которая принимает значение из столбца total_income и 
# исходя из распределения доходов присваивает определенную категорию
def categories_money(money):
    if money<=money_cat['25%']:
        return 'низкий доход'
    if money_cat['25%']<=money<=money_cat['50%']:
        return 'средний доход'
    if money_cat['50%']<=money<=money_cat['75%']:
        return 'доход выше среднего'
    if money_cat['75%']<=money:
        return 'высокий доход'
           

Чтобы было понятно, доходы я разделил на **4** категории:
1. Низкий доход (меньше 107 тыс./месяц), то что в money_cat до 25%
2. Средний доход (от 107 до 145 тыс./месяц), то что в money_cat до 50%
3. Доход выше среднего (от 145 до 195 тыс./месяц)
4. Высокий доход (от 195 тыс./месяц)

In [44]:
# создаем новый столбец 'money_category' и применяем функцию
data['money_category'] = data['total_income'].apply(categories_money)


In [45]:
display(data.head(20))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,target_category,money_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"процессы, связанные с недвижимостью",высокий доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"процессы, связанные с автомобилем",средний доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"процессы, связанные с недвижимостью",доход выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,получение образования,высокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,проведение свадьбы,доход выше среднего
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"процессы, связанные с недвижимостью",высокий доход
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"процессы, связанные с недвижимостью",высокий доход
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,получение образования,средний доход
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,проведение свадьбы,низкий доход
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"процессы, связанные с недвижимостью",средний доход


**Вывод**

В данном разделе мы для каждой категории создали список определенных ключевых значений. Написали функцию **categories**, которая принимает значение в колонке 'purpose' и после лемматизации присваивает ему определенную категорию, сохраняя ее в новой колонке 'target_category'. 
Разделили доходы населения на 4 категории, написали функцию **categories_money**, которая исходя из значения дохода присваивает одну из четырех категорий. Результат сохраняется в колонке 'money_category'.

Для работы функции с определенным столбцом был использован метод apply(), работа которого представлена выше в проекте.

Результатом работы стали новые столбцы в датафреймы с отображением категории цели и категории доходов заявителей

## Ответим на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

* Для начала я отфильтровал значения в колонке `children`, чтобы посмотреть, как часто встречается та или иная категория заявителей в банк с детьми или без

In [46]:
# создаем series по количеству заявителей с детьми или без
data['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

* Далее с помощью метода pivot_table() мы создаем сводную таблицу, где отражено следующее:

    1. `count debt` - показывает количество заявителей с детьми или без;
    2. `mean debt` - показывает какая доля из этой категории имеет задолженность по возврату кредита.

In [47]:
# создаем первую сводную таблицу 
data_pivot = data.pivot_table(index=['children'], values='debt', aggfunc=['count', 'mean'])
data_pivot

,count,mean
,debt,debt
children,,
0,14091,0.075438
1,4855,0.091658
2,2128,0.094925
3,330,0.081818
4,41,0.097561
5,9,0.000000


* Т.е. из 14 тыс. заявителей без детей 7,5% имеет задолженность перед банком

In [48]:
# для того, чтобы видеть наглядный процент должников умножим значение колонки 'mean_debt' на 100 

data_pivot['percentage_of_arrears,%'] = data_pivot['mean']['debt']*100
data_pivot

,count,mean,"percentage_of_arrears,%"
,debt,debt,
children,,,
0,14091,0.075438,7.543822
1,4855,0.091658,9.165808
2,2128,0.094925,9.492481
3,330,0.081818,8.181818
4,41,0.097561,9.756098
5,9,0.000000,0.000000


**Вывод**

- По результатам сводной таблицы наблюдаем следующие результаты:
    
    1. Явно видно, что клиенты банка, не имеющие детей, лучше закрывают кредиты по сравнению с теми у кого дети есть. Думаю, это связано с тем, что свободных денег у этой категории больше для того, чтобы закрыть кредит в срок или досрочно. Также у них больше сводобного времени, а значит недостающую сумму можно успеть заработать, не отвлекаясь на детей).

    2. Малодетная категория населения (1-2 ребенка) хуже справляется со своими обязанностями перед банком. Возможно это связано с тем, что люди не до конца готовы к рождению первого или второго ребенка и не совсем правильно распределяют свой бюджет. Плюс различные форсмажорные обстоятельства, связанные с детьми, которые так или иначе влияют на выполнение кредитных обязательств в срок.

    3. Данные показывают, что многодетные семьи (от 3х детей) намного реже берут кредит. Интересным остается то, что многодетная семья с тремя детьми справляется с задачей выплата кредита лучше, чем малодетная. Предполагаю, что эта категория на опыте лучше распределяет свой бюджет, также можно сэкономить на вещах для малышей, кроватке или коляске, которые переходят от старшего ребенка. Возможно в этом помогают некоторые льготы, положенные многодетным семьям в нашей стране, которые помогают отложить часть денег и вернуть кредит вовремя.

    4. Самый большой процент должников среди заявителей с 4мя детьми. Четверых детей вырастить конечно нелегко, и несмотря на различные льготы и хоть какую-то помощь государства, возможно у них чаще возникают материальные проблемы, которые не позволяют платить в срок и полностью.

    5. Заявители, имеющие 5 детей выплачивают кредит вовремя, но здесь данных очень мало, всего 9 человек, поэтому предположить почему так происходит не представляется возможным. Возможно со временем получится вернуться к этой категории граждан и провести анализ еще раз для того, чтобы отследить изменения.


### Есть ли зависимость между семейным положением и возвратом кредита в срок?

* Для начала отфильтруем значения в колонке `family_status`, чтобы посмотреть, как часто встречается та или иная категория заявителей.

In [49]:
# создаем series по семейному положению
data['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [50]:
# создаем  сводную таблицу по количеству заявителей и доли должников
data_pivot_1 = data.pivot_table(index=['family_status'], values='debt', aggfunc=['count', 'mean'])
data_pivot_1

,count,mean
,debt,debt
family_status,,
Не женат / не замужем,2810,0.097509
в разводе,1195,0.071130
вдовец / вдова,959,0.065693
гражданский брак,4151,0.093471
женат / замужем,12339,0.075452


In [51]:
# для того, чтобы видеть наглядный процент должников умножим значение колонки 'mean_debt' на 100 
data_pivot_1['family_status_arrears,%'] = data_pivot_1['mean']['debt']*100
data_pivot_1

,count,mean,"family_status_arrears,%"
,debt,debt,
family_status,,,
Не женат / не замужем,2810,0.097509,9.750890
в разводе,1195,0.071130,7.112971
вдовец / вдова,959,0.065693,6.569343
гражданский брак,4151,0.093471,9.347145
женат / замужем,12339,0.075452,7.545182


**Вывод**

Результаты анализа показывают следующую зависимость:
* Вдовы и вдовцы имеют меньшую кредитную задолженность 
* Люди, находящиеся в браке, а также те, кто уже разведен, относительно одинаково выплачивают кредит, немного больше 7% имеют кредитную задолженность перед банком
* Категория людей, состоящий в гражданском браке, а также неженатые/незамужние показывают наибольший процент задолжности по кредиту. Почти 10% из каждой категории каким-то образом оказывались должником банка.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [52]:
# создаем series по категории доходы заявителей
data['money_category'].value_counts()

средний доход          6415
высокий доход          5364
низкий доход           5364
доход выше среднего    4311
Name: money_category, dtype: int64

In [53]:
# создаем  сводную таблицу по количеству заявителей и доли должников
data_pivot_2 = data.pivot_table(index=['money_category'], values='debt', aggfunc=['count', 'mean'])
data_pivot_2

,count,mean
,debt,debt
money_category,,
высокий доход,5364,0.071402
доход выше среднего,4311,0.089074
низкий доход,5364,0.079605
средний доход,6415,0.085269


In [54]:
# для того, чтобы видеть наглядный процент должников умножим значение колонки 'mean_debt' на 100 
data_pivot_2['money_category_arrears,%'] = data_pivot_2['mean']['debt']*100
data_pivot_2

,count,mean,"money_category_arrears,%"
,debt,debt,
money_category,,,
высокий доход,5364,0.071402,7.140194
доход выше среднего,4311,0.089074,8.907446
низкий доход,5364,0.079605,7.960477
средний доход,6415,0.085269,8.526890


**Вывод**

* На мой взгляд резкой зависимости между уровнем дохода и возвратом кредита в срок не наблюдается. Конечно видно, что люди с высоким доходом имеют меньший процент задолжности перед банком, но в среднем 8% людей не успевают вернуть кредит в срок, независимо от их финансового состояния

### Как разные цели кредита влияют на его возврат в срок?

In [55]:
# создаем series по категории доходы заявителей
data['target_category'].value_counts()

процессы, связанные с недвижимостью    10811
процессы, связанные с автомобилем       4306
получение образования                   4013
проведение свадьбы                      2324
Name: target_category, dtype: int64

In [56]:
# создаем  сводную таблицу по количеству заявителей и доли должников
data_pivot_3 = data.pivot_table(index=['target_category'], values='debt', aggfunc=['count', 'mean'])
data_pivot_3

,count,mean
,debt,debt
target_category,,
получение образования,4013,0.092200
проведение свадьбы,2324,0.080034
"процессы, связанные с автомобилем",4306,0.093590
"процессы, связанные с недвижимостью",10811,0.072334


In [57]:
# для того, чтобы видеть наглядный процент должников умножим значение колонки 'mean_debt' на 100 
data_pivot_3['target_category_arrears,%'] = data_pivot_3['mean']['debt']*100
data_pivot_3

,count,mean,"target_category_arrears,%"
,debt,debt,
target_category,,,
получение образования,4013,0.092200,9.220035
проведение свадьбы,2324,0.080034,8.003442
"процессы, связанные с автомобилем",4306,0.093590,9.359034
"процессы, связанные с недвижимостью",10811,0.072334,7.233373


**Вывод**

* Люди, которые берут кредит для покупки недвижимости под разные задачи лучше выплачивают кредит по сравнению со всеми остальными категориями. На мой взгляд это может быть связано с тем, что данный кредит оформляется под залог самой квартиры или какого-то другого имущества и люди ведут себя более ответственно.

* Среди тех, кто берет кредит на свадьбу около 8% имеют задолженность перед банком по оплате такого кредита.

* Люди, получившие кредит на образование или авто менее ответственно подходят к договорным обязательствам с банком и имеют наибольший процент задолженностей.

## Общий вывод

В данной работе проведено исследование надёжности заёмщиков.
На вход были получены данные от банка — статистика о платёжеспособности клиентов.

1. Были изучены полученные данные. Первоначальная таблица состояла из 12 столбцов и содержала 3 типа данных.

2. Далее  данные были обработаны: проведена работа с пропусками, заменены значения ежемесячного дохода и трудового стажа, количество детей приведено к реальным показателям; трудовой стаж приведен к целочисленному типу данных. Произведена обработка дубликатов посредством их удаления, так как они составили всего 0,33% от общего числа данных. Проведена лемматизация, которая понадобилась для того, чтобы легче провести категоризацию данных и разбить множество целей кредита на 4 основные. По результатам вышеперечисленных процессов были получены сводные таблицы для ответа на вопросы кредитного отдела банка.

* Лучше всех закрывают кредит люди, не имеющие детей
* Женатые, разведенные или овдовевшие заявители имеют низкий процент задолженности по сравнению с холостыми и живущими в гражданском браке
* Люди с высоким и низким доходом лучше справляются со своими кредитными обязанностями
* Наиболее рискованные цели - это кредит на образование и автомобиль, к выплате кредита за недвижимость или проведение свадьбы люди подходят более ответственно.

**Лучший заемщик для банка** - это человек с высоким доходом, который желает приобрести недвижимость, не имеющий детей и находящийся в разводе или овдовевший.
